<a href="https://colab.research.google.com/github/MOHOAzure/AI_Paint_Online/blob/main/Colab/LoRA_training_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LoRA 模型訓練
1. 執行階段->變更執行階段類型->硬體加速器選擇GPU
2. 掛接雲端硬碟
3. 將資料集放在/content/drive/MyDrive/LoRA_training/datasets
4. 建立環境
5. 訓練參數設置
6. 開始訓練LoRA

詳細步驟請參考[巴哈教學](https://home.gamer.com.tw/artwork.php?sn=5652683)

In [ ]:
#@title 掛接雲端硬碟
from google.colab import drive
drive.mount('/content/drive')
!cd /content/drive/MyDrive/
!mkdir /content/drive/MyDrive/LoRA_training/
!mkdir /content/drive/MyDrive/LoRA_training/datasets
!mkdir /content/drive/MyDrive/LoRA_training/output
!mkdir /content/drive/MyDrive/LoRA_training/log

In [ ]:
#@title 建立環境
!sudo apt-get update -y && sudo apt-get install python3.10 python3.10-dev python3.10-distutils libpython3.10-dev python3.10-venv

# Link python3 to 3.10.
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 2

# Install pip.
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.9

# Install your dependencies. Pin IPython and traitlets versions, as traitlets version 5.8 contains a breaking change that prevents google-colab server extension from working properly.
# Some of these packages may not be required.
!python3 -m pip install ipython==7.9.0 traitlets==5.7.1 jupyter psutil setuptools ipython_genutils ipykernel jupyter_console prompt_toolkit httplib2 astor

# Copy the already installed `google-colab` package over. It depends on an _old version_ of pandas, so building the wheel won't work.
!ln -s /usr/local/lib/python3.8/dist-packages/google \
       /usr/local/lib/python3.10/dist-packages/google

# Reinstall jupyter server extension.
!jupyter serverextension list
!jupyter notebook --config=/usr/local/etc/jupyter/jupyter_notebook_config.json

#Reinstall pip
!python3 -m ensurepip --upgrade

%cd /content
!git clone https://github.com/kohya-ss/sd-scripts
!git checkout 53d60543e59d6fdf3a6b5d0d15019487d2e415db
%cd /content/sd-scripts

!pip install torch==1.12.1+cu116 torchvision==0.13.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116
!pip install --upgrade -r requirements.txt
!pip install -U -I --no-deps https://github.com/C43H66N12O12S2/stable-diffusion-webui/releases/download/linux/xformers-0.0.14.dev0-cp310-cp310-linux_x86_64.whl


In [ ]:
#@title 訓練參數設置

#@markdown ##專案名稱
project_name = "IWSK" #@param {type:"string"}
train_data_dir = "/content/drive/MyDrive/LoRA_training/datasets/" + project_name
output_dir = "/content/drive/MyDrive/LoRA_training/output/" + project_name
logging_dir = "/content/drive/MyDrive/LoRA_training/log"
#@markdown 將資料集資料夾放在 `/content/drive/MyDrive/LoRA_training/datasets` ，資料夾名稱為 `project_name`。

#@markdown ##訓練相關參數
#@markdown Base Model路徑
model_path = "/content/drive/MyDrive/stable-diffusion/models/'sd-v1-4.ckpt'" #@param {type:"string"}
#@markdown 圖片解析度
width = 512 #@param {type:"slider", min:256, max:1024, step:64}
height = 512 #@param {type:"slider", min:256, max:1024, step:64}
#@markdown batch大小
train_batch_size = 4 #@param {type:"slider", min:1, max:8, step:1}
#@markdown 最大訓練epochs
max_train_epochs = 40 #@param {type:"integer"}
#@markdown 每n個epochs存檔一次
save_every_n_epochs = 5 #@param {type:"integer"}
#@markdown LoRA模型大小
network_dim = 32 #@param {type:"slider", min:1, max:128, step:1}
#@markdown 打亂標籤
use_shuffle_caption = True #@param {type:"boolean"}

#@markdown ##學習率
lr = 1e-4 #@param {type:"number"}
unet_lr = 1e-4 #@param {type:"number"}
text_encoder_lr = 1e-5 #@param {type:"number"}
lr_scheduler = "cosine_with_restarts" #@param ["constant", "constant_with_warmup", "cosine", "cosine_with_restarts", "linear", "polynomial"]
#@markdown 只有在 `lr_scheduler` 為constant_with_warmup時需要填寫 `lr_warmup_steps` 。
lr_warmup_steps = 0 #@param {type:"integer"}

#@markdown ##輸出模型
save_model_as = "safetensors" #@param ["safetensors", "ckpt", "pt"]

if (use_shuffle_caption):
  shuffle_caption = "--shuffle_caption"
else:
  shuffle_caption = ""
  

In [ ]:
#@title 開始訓練LoRA
%cd /content/sd-scripts
!accelerate launch \
  --mixed_precision 'fp16' \
  --num_cpu_threads_per_process=2 \
  "train_network.py" \
  --cache_latents \
  --enable_bucket \
  --use_8bit_adam \
  --mem_eff_attn \
  --pretrained_model_name_or_path=$model_path \
  --train_data_dir=$train_data_dir \
  --resolution=$width,$height \
  --output_dir=$output_dir \
  --train_batch_size=$train_batch_size \
  --lr_scheduler=$lr_scheduler \
  --lr_warmup_steps=$lr_warmup_steps \
  --max_train_epochs=$max_train_epochs \
  --use_8bit_adam \
  --mixed_precision=fp16 \
  --save_every_n_epochs=$save_every_n_epochs \
  --seed=4242 \
  --save_precision=fp16 \
  --logging_dir=$logging_dir \
  --save_model_as=$save_model_as \
  --network_module=networks.lora \
  --text_encoder_lr=$text_encoder_lr \
  --unet_lr=$unet_lr \
  --learning_rate=$lr \
  --network_dim=$network_dim \
  --network_alpha=16 \
  --clip_skip=2 \
  --caption_extension=".txt" \
  --output_name=$project_name \
  $shuffle_caption

  # --network_alpha=$network_dim \

In [ ]:
#@title Setup Tool to use models on Drive

# (optional: check GPU)
!nvidia-smi

# define working directory
root_dir = "/content"
tool_dir = "/content/stable-diffusion-webui"

%cd $root_dir

# Load Google Drive for my models
from google.colab import drive
drive.mount('/content/drive')

# 685f96: The version most commonly used to merge models
version = 'latest' #@param ["latest", "685f9631b56ff8bd43bce24ff5ce0f9a0e9af490"] {allow-input: true}

def use_cuda():
  # use cuda instead of cpu for large model (e.g., 7G)
  !sed -i 's/weight_load_location = None if cmd_opts.lowram else "cpu"/weight_load_location = None if cmd_opts.lowram else "cuda"/g' /content/stable-diffusion-webui/modules/shared.py

def install_dep():
  # install xformers
  %cd /content
  !pip install xformers
  # !python -m xformers.info

def install_extentions():
  %cd {tool_dir}/extensions
  
  !git clone https://github.com/7eu7d7/DreamArtist-sd-webui-extension.git
  !git clone https://github.com/bbc-mc/sdweb-merge-board
  !git clone https://github.com/tkalayci71/embedding-inspector
  !git clone https://github.com/toriato/stable-diffusion-webui-wd14-tagger
  !git clone https://github.com/bbc-mc/sdweb-merge-block-weighted-gui
  !git clone https://github.com/Malisius/booru2prompt.git
  !git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete.git
  !git clone https://github.com/toshiaki1729/stable-diffusion-webui-dataset-tag-editor
  # !git clone https://github.com/adieyal/sd-dynamic-prompts
  # !git clone https://github.com/kousw/stable-diffusion-webui-daam
  # !git clone 

  # LoRA  
  !git clone https://github.com/cloneofsimo/lora.git && sed -i 's/functools.cache/functools.lru_cache(maxsize=None)/g' /content/lora/lora_diffusion/xformers_utils.py && pip install /content/lora
  !pip install accelerate bitsandbytes
  !git clone https://github.com/kohya-ss/sd-webui-additional-networks

def set_default_prompt_and_neg():
  # TODO
  pass

# Install tool: stable-diffusion-webui
repo = "https://github.com/AUTOMATIC1111/stable-diffusion-webui.git"
!git clone $repo

# time machine: commit or branch
if version:
  !git checkout $version

install_dep()

use_cuda()

install_extentions()

In [ ]:
#@title move training result
!cp /content/drive/MyDrive/LoRA_training/output/IWSK/IWSK.safetensors /content/stable-diffusion-webui/extensions/sd-webui-additional-networks/models/lora/

In [ ]:
#@title Launch
%cd {tool_dir}

model_dir = "/content/drive/MyDrive/stable-diffusion/models"

# security
account = "me" #@param {type:"string"}
pwd = "sb@CCL00" #@param {type:"string"}

# use param to run tool
# notice: some param are only available at later versions
arg = f"--enable-insecure-extension-access --skip-version-check --share --gradio-debug --gradio-auth {account}:{pwd} --disable-safe-unpickle --ckpt-dir {model_dir}"

!COMMANDLINE_ARGS="{arg}" REQS_FILE="requirements.txt" python launch.py


  0% 0/40 [00:00<?, ?it/s]
  2% 1/40 [00:00<00:32,  1.20it/s]
  5% 2/40 [00:01<00:31,  1.19it/s]
  8% 3/40 [00:02<00:30,  1.19it/s]
 10% 4/40 [00:03<00:30,  1.19it/s]
 12% 5/40 [00:04<00:29,  1.19it/s]
 15% 6/40 [00:05<00:28,  1.19it/s]
 18% 7/40 [00:05<00:27,  1.18it/s]
 20% 8/40 [00:06<00:27,  1.18it/s]
 22% 9/40 [00:07<00:26,  1.19it/s]
 25% 10/40 [00:08<00:25,  1.18it/s]
 28% 11/40 [00:09<00:24,  1.18it/s]
 30% 12/40 [00:10<00:23,  1.19it/s]
 32% 13/40 [00:10<00:22,  1.18it/s]
 35% 14/40 [00:11<00:22,  1.18it/s]
 38% 15/40 [00:12<00:21,  1.18it/s]
 40% 16/40 [00:13<00:20,  1.18it/s]
 42% 17/40 [00:14<00:19,  1.18it/s]
 45% 18/40 [00:15<00:18,  1.18it/s]
 48% 19/40 [00:16<00:17,  1.18it/s]
 50% 20/40 [00:16<00:17,  1.18it/s]
 52% 21/40 [00:17<00:16,  1.18it/s]
 55% 22/40 [00:18<00:15,  1.17it/s]
 57% 23/40 [00:19<00:14,  1.18it/s]
 60% 24/40 [00:20<00:13,  1.18it/s]
 62% 25/40 [00:21<00:12,  1.17it/s]
 65% 26/40 [00:22<00:11,  1.17it/s]
 68% 27/40 [00:22<00:11,  1.17it/s]
 70% 28/40